# Cenário 2: 16 qubits e 1 alvo

- Objetivo: criar circuito com 16 qubits com o alvo de maior valor possível $|111...1\rangle$ e comparar com a busca linear clássica 

Neste cenário, usamos:
- Número de qubits: $n = 16$
- Espaço de busca: $N = 2^n = 65.536$
- Número de alvos: $M = 1$
- Alvo: $|1111111111111111\rangle$

O número ideal de iterações de Grover é:
$$
k \approx \left\lfloor \frac{\pi}{4}\sqrt{\frac{N}{M}} \right\rfloor
$$

## Implementação do Algoritmo de Grover para o Cenário 2

In [11]:
# =================================================
# Funções auxiliares para a construção do algoritmo
# (Cenário 2: n=16, 1 alvo |111...1>)
# =================================================

import math
from qiskit import QuantumCircuit

# ----------------------------
# Configuração fixa do Cenário 2
# ----------------------------
SCENARIO_ID = 2
N_QUBITS = 16
SHOTS_DEFAULT = 1024
TARGET_DEFAULT = "1" * N_QUBITS   # |111...1>
M_TARGETS = 1


def grover_iterations(n: int, m: int) -> int:
    """
    Calcula o número ideal de iterações do algoritmo de Grover.

    A aproximação usada é:

    $$
    k \\approx \\left\\lfloor \\frac{\\pi}{4}\\sqrt{\\frac{N}{M}} \\right\\rfloor
    $$

    onde $$N = 2^n$$ e $$M$$ é o número de alvos.

    Parameters
    ----------
    n : int
        Número de qubits.
    m : int
        Número de estados-alvo (soluções).

    Returns
    -------
    int
        Número de iterações k.
    """
    if n < 1:
        raise ValueError("n deve ser >= 1.")
    if m < 1:
        raise ValueError("m deve ser >= 1.")
    N = 2**n
    return int(math.floor((math.pi / 4) * math.sqrt(N / m)))


def oracle_phase_single_target(target: str, n: int = N_QUBITS) -> QuantumCircuit:
    """
    Cria um oráculo de fase que aplica -1 SOMENTE ao estado |target>.

    Implementação "na mão":
    - Aplica X nos qubits onde o alvo tem bit 0, para mapear |target> -> |11...1>
    - Aplica um MCZ (multi-controlled Z) em |11...1>
      MCZ é implementado como: H(no último) + MCX + H(no último)
    - Desfaz os X

    Parameters
    ----------
    target : str
        String binária com n bits. Para o cenário 2, o padrão é "1"*16.
    n : int
        Número de qubits. Por padrão, n=16 (cenário 2).

    Returns
    -------
    QuantumCircuit
        Circuito do oráculo de fase reutilizável.

    Notes
    -----
    - Para n>=2, usamos MCX (multi-controlled X). Isso pode tornar o circuito grande,
      e o Qiskit pode "paginar" a visualização no draw (é o mesmo circuito, só dividido).
    """
    if len(target) != n:
        raise ValueError(f"target deve ter exatamente {n} bits.")
    if any(b not in "01" for b in target):
        raise ValueError("target deve conter apenas '0' e '1'.")

    qc = QuantumCircuit(n, name=f"Oracle({target})")

    # Mapear |target> -> |11..1| aplicando X onde target tem 0
    for i, bit in enumerate(target):
        if bit == "0":
            qc.x(i)

    # Aplicar -1 em |11..1| via MCZ:
    # MCZ = H(last) · MCX(controls=0..n-2, target=n-1) · H(last)
    qc.h(n - 1)
    controls = list(range(n - 1))
    qc.mcx(controls, n - 1)
    qc.h(n - 1)

    # Desfazer mapeamento
    for i, bit in enumerate(target):
        if bit == "0":
            qc.x(i)

    return qc


def diffuser(n: int = N_QUBITS) -> QuantumCircuit:
    """
    Cria o operador de difusão (inversão sobre a média) para n qubits.

    Decomposição padrão:

        D = H^{⊗n} · X^{⊗n} · MCZ · X^{⊗n} · H^{⊗n}

    e MCZ é implementado como:

        MCZ = H(no último qubit) · MCX(controles=0..n-2, alvo=n-1) · H(no último qubit)

    Parameters
    ----------
    n : int
        Número de qubits do registrador de busca. Por padrão, n=16 (cenário 2).

    Returns
    -------
    QuantumCircuit
        Circuito do difusor reutilizável.
    """
    if n < 2:
        raise ValueError("Para o cenário 2, espera-se n >= 2 (n=16).")

    qc = QuantumCircuit(n, name=f"Diffuser({n}q)")

    qc.h(range(n))
    qc.x(range(n))

    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)

    qc.x(range(n))
    qc.h(range(n))

    return qc


def build_grover_circuit(
    n: int = N_QUBITS,
    target: str = TARGET_DEFAULT,
    shots: int = SHOTS_DEFAULT
) -> tuple[QuantumCircuit, int]:
    """
    Monta o circuito completo do Grover (preparação + k iterações + medição),
    parametrizado para o Cenário 2 por padrão (n=16, alvo=|111...1>).

    Parameters
    ----------
    n : int
        Número de qubits. Padrão: 16.
    target : str
        Estado alvo em binário (n bits). Padrão: "1"*16.
    shots : int
        Apenas para manter o parâmetro do cenário organizado (não executa aqui).

    Returns
    -------
    (QuantumCircuit, int)
        O circuito final com medições e o número de iterações k.
    """
    if len(target) != n:
        raise ValueError("target deve ter exatamente n bits.")

    m = M_TARGETS
    k = grover_iterations(n, m)

    oracle = oracle_phase_single_target(target, n=n)
    diff = diffuser(n)

    qc = QuantumCircuit(n, n, name=f"Grover(n={n}, target={target})")

    # 1) Preparação: superposição uniforme
    qc.h(range(n))

    # 2) k iterações: (oráculo + difusor)
    for _ in range(k):
        qc.compose(oracle, inplace=True)
        qc.compose(diff, inplace=True)

    # 3) Medições
    qc.measure(range(n), range(n))

    return qc, k


def print_scenario_summary(
    scenario_id: int = SCENARIO_ID,
    n: int = N_QUBITS,
    target: str | list[str] = TARGET_DEFAULT,
    shots: int = SHOTS_DEFAULT,
    k: int | None = None
) -> None:
    """
    Imprime um resumo padronizado do cenário de execução do algoritmo de Grover.

    Por padrão, imprime o Cenário 2 (n=16, alvo=|111...1>).

    Parameters
    ----------
    scenario_id : int
        Identificador do cenário (padrão: 2).
    n : int
        Número de qubits (padrão: 16).
    target : str or list[str]
        Estado alvo (binário) ou lista de estados alvo.
    shots : int
        Número de medições realizadas (padrão: 1024).
    k : int or None
        Número de iterações de Grover aplicadas. Se None, será calculado para M=1.
    """
    if k is None:
        k = grover_iterations(n, M_TARGETS)

    if isinstance(target, list):
        target_str = ", ".join(f"|{t}>" for t in target)
    else:
        target_str = f"|{target}>"

    print(
        f"Cenário {scenario_id} -> "
        f"n={n}, "
        f"alvo(s)={target_str}, "
        f"shots={shots}, "
        f"iterações k={k}"
    )

In [21]:
# =========================================
# Construção do Circuito Quântico de Grover
# =========================================

from qiskit.visualization import circuit_drawer
from qiskit_aer import AerSimulator
from qiskit import transpile

# Construir o circuito (caso ainda não tenha sido construído nesta célula)
grover_circuit, k = build_grover_circuit()

print_scenario_summary(k=k)

# -------------------------------------------------
# 1) Renderização em TEXTO (recomendado para n=16)
#    Garante que seja exibido UM ÚNICO circuito
# -------------------------------------------------
print("\n=== Circuito lógico (alto nível | texto) ===")
grover_circuit.draw(output="text", fold=-1)

# -------------------------------------------------
# 2) Renderização após transpilation (o circuito REAL executado)
# -------------------------------------------------
# backend = AerSimulator()
# transpiled_circuit = transpile(grover_circuit, backend, optimization_level=1)

# print("\n=== Circuito transpilado (texto) ===")
# print(transpiled_circuit.draw(output="text", fold=200))

Cenário 2 -> n=16, alvo(s)=|1111111111111111>, shots=1024, iterações k=201

=== Circuito lógico (alto nível | texto) ===


┌───┐          ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐

In [22]:
# -------------------------------------------------
# 3) (Opcional) Renderização em imagem (MPL)
#    Atenção: pode ser paginado pelo Qiskit se ficar muito grande
# -------------------------------------------------
# Descomente apenas se quiser visualizar graficamente
circuit_drawer(
    grover_circuit,
    output="mpl",
    fold=-1
)

ValueError: Image size of 169618x1454 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 169618x1454.83 with 1 Axes>